# AutoGluon California Housing

Regression example using AutoGluon Tabular. Use Kaggle data (preferred) or sklearn fetch fallback. Keep data out of git; run in Colab/local `.venv`.

## 1) Install & imports

In [1]:
# If in Colab, uncomment:
# !pip install -q autogluon.tabular kaggle

import os, zipfile
from pathlib import Path
import pandas as pd
from autogluon.tabular import TabularPredictor
from sklearn.datasets import fetch_california_housing


## 2) Load data (Kaggle or sklearn fallback)

In [2]:
data_dir = Path('data')
data_dir.mkdir(exist_ok=True)

train_path = data_dir / 'california_housing_train.csv'
if train_path.exists():
    df = pd.read_csv(train_path)
else:
    # fallback to sklearn dataset
    cali = fetch_california_housing(as_frame=True)
    df = cali.frame
    df.rename(columns={'MedHouseVal': 'median_house_value'}, inplace=True)

SAMPLE_FRACTION = float(os.environ.get('SAMPLE_FRACTION', '0.1'))  # adjust for speed
if SAMPLE_FRACTION < 1.0:
    df = df.sample(frac=SAMPLE_FRACTION, random_state=42)

print(df.head())
print('Shape:', df.shape)


      MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
996   3.1333      30.0  5.925532   1.131206       966.0  3.425532     36.51   
468   2.3355      18.0  5.711722   1.059809      1868.0  2.234450     33.97   
6078  3.3669      29.0  4.589878   1.076789      1071.0  1.869110     34.15   
2127  3.8750      46.0  4.000000   1.000000        59.0  4.538462     33.12   
2080  4.3482       9.0  5.792453   1.103774       409.0  1.929245     35.36   

      Longitude  median_house_value  
996     -119.65               1.000  
468     -117.01               1.188  
6078    -118.37               3.761  
2127    -117.11               2.000  
2080    -119.06               0.952  
Shape: (619, 9)


## 3) Train/test split

In [3]:
from sklearn.model_selection import train_test_split

label = 'median_house_value'
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print('Train shape:', train_df.shape, 'Test shape:', test_df.shape)


Train shape: (495, 9) Test shape: (124, 9)


## 4) Fit AutoGluon predictor

In [4]:
predictor = TabularPredictor(label=label, eval_metric='rmse', path='ag_california_models')
predictor.fit(train_df, presets='best_quality', time_limit=60)  # adjust time_limit


Verbosity: 2 (Standard Logging)


=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 25.1.0: Mon Oct 20 19:32:41 PDT 2025; root:xnu-12377.41.6~2/RELEASE_ARM64_T6000
CPU Count:          8
Memory Avail:       4.68 GB / 16.00 GB (29.2%)
Disk Space Avail:   268.95 GB / 460.43 GB (58.4%)


Presets specified: ['best_quality']


Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)


Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1


...

	-0.5538	 = Validation score   (-root_mean_squared_error)


	0.0s	 = Training   runtime


	0.0s	 = Validation runtime


AutoGluon training complete, total runtime = 44.66s ... Best model: WeightedEnsemble_L3 | Estimated inference throughput: 1725.4 rows/s (62 batch size)


TabularPredictor saved. To load, use: predictor = TabularPredictor.load("ag_california_models")


## 5) Leaderboard & feature importance

In [5]:
lb = predictor.leaderboard(test_df, silent=True)
fi = predictor.feature_importance(test_df)
print(lb.head())
print(fi.head())


Computing feature importance via permutation shuffling for 8 features using 124 rows with 5 shuffle sets...


	8.11s	= Expected runtime (1.62s per shuffle set)


	1.65s	= Actual runtime (Completed 5 of 5 shuffle sets)


                    model  score_test  score_val              eval_metric  \
0         LightGBM_BAG_L1   -0.607289  -0.605915  root_mean_squared_error   
1         CatBoost_BAG_L2   -0.607664  -0.594196  root_mean_squared_error   
2  RandomForestMSE_BAG_L1   -0.608801  -0.642286  root_mean_squared_error   
3     WeightedEnsemble_L2   -0.609262  -0.556845  root_mean_squared_error   
4         CatBoost_BAG_L1   -0.610340  -0.556902  root_mean_squared_error   

   pred_time_test  pred_time_val   fit_time  pred_time_test_marginal  \
0        0.020950       0.004528   9.848326                 0.020950   
1        0.148135       0.125567  29.778347                 0.004811   
2        0.021190       0.038374   0.167362                 0.021190   
3        0.034923       0.009922  17.744485                 0.000926   
4        0.013047       0.005262   7.891983                 0.013047   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.004528       

## 6) Evaluate and predict

In [6]:
performance = predictor.evaluate(test_df)
print('Performance:', performance)

sample_preds = predictor.predict(test_df.head(5))
print(sample_preds)


Performance: {'root_mean_squared_error': -0.6110734200982806, 'mean_squared_error': -0.37341072475060977, 'mean_absolute_error': -0.42706863899784697, 'r2': 0.6961352847777107, 'pearsonr': 0.8345130842939088, 'median_absolute_error': -0.32182136058807376}


167     0.941218
1578    4.124524
4597    3.525582
625     2.088022
2857    1.316015
Name: median_house_value, dtype: float32
